# Building an SST-based Autoencoder for Interpretation of ENSO


In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

All of the data is in `.nc`-formatted files from NOAA; we'll use the `xarray` utility to deal with the high dimensional data. 

We have to start by judiciously building up the data set from the 2000 files, because we can't open them all at once.
We split the files into lists of 30 (There are 1957 total files). We load the data by aggregating each month in the dataset into a single .nc file. We have to do this becuase of Python's limit on the number of files we can open (1957 exceeds this limit).  

In [ ]:
os.remove("data/aggregated.nc")
data_files = os.listdir("data/")
n = 30
prtn_dfs = [data_files[i:i + n] for i in xrange(0, len(data_files), n)]
first = prtn_dfs[0]
cbd = xr.concat([xr.open_dataset("./data/" + j) for j in first], 'time')
prtn_dfs = prtn_dfs[1:]

for prtn in prtn_dfs:
    i_cbd = xr.concat([xr.open_dataset("./data/" + j) for j in prtn], 'time')
    cbd = xr.concat([cbd, i_cbd], 'time')
cbd.to_netcdf('./data/aggregated.nc')

We then select our SST data variable, and slice the data between 6S - 6N in latitude, and 120W - 170W in longitude. 

Now we have our formatted data. Next, we reshape the 7x26 matrix for each data point in time into 182 features. 


In [2]:
ds = xr.open_dataset('./data/aggregated.nc')
nino34 = ds['sst'].sel(lat=slice(-6.0, 6.0), lon=slice(190, 240))
nino34 = nino34[:, 0, :, :]

demean = lambda df: df - df.mean()

#means = nino34.groupby('time.month').apply(np.mean)

nino34 = nino34.groupby('time.month') - nino34.groupby('time.month').apply(np.mean)
nino34 = nino34 / np.max([-1 * np.min(nino34.values), np.max(nino34.values)])

In [18]:
nino34.shape

(1957, 7, 26)

In [3]:
start_test = (nino34.shape[0] - 300)
x_train = nino34[:start_test, :, :].values.astype('float32')\
            .reshape(start_test, nino34.shape[1] * nino34.shape[2])
x_test = nino34[start_test:, :, :].values.astype('float32')\
            .reshape(nino34.shape[0] - start_test, nino34.shape[1] * nino34.shape[2])
print x_train.shape
print x_test.shape

(1657, 182)
(300, 182)


In [ ]:
for i in list(nino34.time.values):
    print i

### Creating the actual Autoencoder

We decide to strip our 182 features down to a much smaller (~3) feature representation of the data. The following creates the autoencoder network. 

In [4]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers


# this is the size of our encoded representations
encoding_dim = 5

# this is our input placeholder
input_layer = Input(shape=(182,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='tanh')(input_layer)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(182, activation='tanh')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_layer, decoded)

rms_prop = optimizers.RMSprop(lr=0.0001)

Using TensorFlow backend.


In [5]:
autoencoder.compile(optimizer=rms_prop, loss='mse')

In [6]:
non_linear_history = autoencoder.fit(x_train, x_train,
                nb_epoch=100,
                batch_size=64,
                validation_data=(x_test, x_test))

Train on 1657 samples, validate on 300 samples
Epoch 1/100
1657/1657 [==============================] - 0s - loss: 0.0542 - val_loss: 0.0586
Epoch 2/100
1657/1657 [==============================] - 0s - loss: 0.0518 - val_loss: 0.0563
Epoch 3/100
1657/1657 [==============================] - 0s - loss: 0.0501 - val_loss: 0.0545
Epoch 4/100
1657/1657 [==============================] - 0s - loss: 0.0486 - val_loss: 0.0529
Epoch 5/100
1657/1657 [==============================] - 0s - loss: 0.0472 - val_loss: 0.0515

In [19]:
encoder = Model(input=input_layer, output=encoded)
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

encoder.predict(x_test[0].reshape((1, 182)))
op = decoder.predict(np.array([[0.2, 0.3, 0.1, 0.5, 0.4]]))
op.reshape((7,26))

array([[  4.62460428e-01,   3.92685384e-01,   4.00436789e-01,
          3.98817688e-01,   3.82267237e-01,   3.47043216e-01,
          3.29789817e-01,   3.63081545e-01,   3.63243312e-01,
          2.97448128e-01,   3.23783159e-01,   2.59977967e-01,
          2.95473367e-01,   2.38998041e-01,   2.53961414e-01,
          1.54148266e-01,   2.08994776e-01,   9.66321528e-02,
          1.10070579e-01,   1.27118424e-01,   6.80684373e-02,
          5.59556633e-02,   7.11826459e-02,   7.50525370e-02,
          7.03324750e-02,  -3.99586521e-02],
       [  4.03680176e-01,   3.46699178e-01,   3.61245841e-01,
          3.20571572e-01,   2.74880379e-01,   3.32840383e-01,
          2.71984041e-01,   3.08339983e-01,   2.68784106e-01,
          2.84701347e-01,   2.49700189e-01,   1.78029224e-01,
          1.94652721e-01,   1.22755758e-01,   1.13952525e-01,
          9.36471373e-02,   1.64479673e-01,   9.34385732e-02,
          9.00179446e-02,   8.82961154e-02,   5.27967373e-03,
          4.27490063e-02,

## EOF Analysis

The first EOF of the data will be the El-Nino mode, and you want your non-linear model to explain more variance than that linear EOF. The reason I haven't asked you to do this is because it's a little complicated to implement, but you can probably figure it out. Keep in mind that EOF analysis ("empirical orthogonal function") is just SVD ("singular value decomposition") analysis applied to timeseries data, so the Python function you will use is likely a SVD function in scipy (scipy.linalg.svd?). 

### Determining Variance Explained

So now we want to determine how much of the variance our model is explaining. We calculate the mode of the data first. 

In [7]:
from scipy.stats import mode

m = mode(np.var(x_train, axis=None))
print(m)
import sys
print(sys.version)

ModeResult(mode=array([ 0.05366411], dtype=float32), count=array([1]))
2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:05:08) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]


Next we use a linear activation function and reduce to 1 variable. We are doing this so that we can reconstruct the output from the one variable (the mode we calculated above) and then compare it to the variance explained in the 

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import optimizers


# this is the size of our encoded representations
encoding_dim = 1

# this is our input placeholder
input_layer = Input(shape=(182,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='linear')(input_layer)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(182, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_layer, decoded)

rms_prop = optimizers.RMSprop(lr=0.0001)

In [ ]:
autoencoder.compile(optimizer=rms_prop, loss='mse')

In [ ]:
linear_history = autoencoder.fit(x_train, x_train,
                    nb_epoch=100,
                    batch_size=64,
                    validation_data=(x_test, x_test))

In [ ]:
#print(history.history['loss'][-1], history.history['val_loss'][-1])
plt.figure()
plt.plot(non_linear_history.history['loss'], label='loss_nonlinear')
plt.plot(non_linear_history.history['val_loss'], label='val_loss_nonlinear')
plt.plot(linear_history.history['loss'], label='loss_linear')
plt.plot(linear_history.history['val_loss'], label='val_loss_linear')
diff = [abs(b-a) for (a,b) in zip(linear_history.history['val_loss'], non_linear_history.history['val_loss'])]
plt.plot(diff, label='diff')
plt.ylim(0)
plt.legend()
plt.show()